In [ ]:
# ==============================================================================
# NOTEBOOK DE TESTES METAMÓRFICOS - NÍVEL 2: ANÁLISE DE COERÊNCIA CAUSAL
# ==============================================================================
# Objetivo: Avaliar se o sistema produz respostas logicamente consistentes a
# transformações de entrada que possuem significado físico ou de negócio,
# explorando o conhecimento conceitual sobre as variáveis de entrada.
# ------------------------------------------------------------------------------

# 📦 Imports
import pandas as pd
import numpy as np
import joblib
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from pathlib import Path
from IPython.display import display

# ------------------------------------------------------------------------------
# 📁 ETAPA 1: SETUP DO AMBIENTE DE TESTE
# ------------------------------------------------------------------------------

try:
    ethanol_path = Path(
        r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/"
        r"MDPI2023-pollution/data/[Etanol] Trajeto Casa-Escola-UFRN/"
        r"trackLog-2023-Feb-13_06-38-49_seg.csv"
    )
    gasoline_path = Path(
        r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/"
        r"MDPI2023-pollution/data/[Gasolina] Trajeto Casa-Escola-UFRN/"
        r"trackLog-2022-Dec-01_06-43-57_qui.csv"
    )

    df_ethanol = pd.read_csv(ethanol_path)
    df_gasoline = pd.read_csv(gasoline_path)

    print("Arquivos de dados carregados com sucesso.")

except FileNotFoundError as e:
    print(f"ERRO: Arquivos de dados não encontrados. {e}")
    raise

# ------------------------------------------------------------------------------
# 🔧 PREPARAÇÃO DOS DADOS
# ------------------------------------------------------------------------------

def preparar_dados(df):
    """
    Padroniza nomes de colunas e calcula aceleração a partir da velocidade,
    preservando coerência temporal entre amostras.
    """
    df_copy = df.copy()
    df_copy.columns = df_copy.columns.str.strip()

    col_map = {
        "Speed (OBD)(km/h)": "Speed(OBD)(km/h)",
        "Latitude ": "Latitude",
        "Longitude ": "Longitude"
    }

    df_copy.rename(
        columns={k: v for k, v in col_map.items() if k in df_copy.columns},
        inplace=True
    )

    if "Speed(OBD)(km/h)" in df_copy.columns:
        speeds_ms = df_copy["Speed(OBD)(km/h)"].fillna(0).values * 1000 / 3600
        df_copy["Acceleration"] = np.diff(speeds_ms, prepend=speeds_ms[0])

    return df_copy


df_ethanol = preparar_dados(df_ethanol)
df_gasoline = preparar_dados(df_gasoline)

# ------------------------------------------------------------------------------
# 🤖 CARREGAMENTO DOS MODELOS
# ------------------------------------------------------------------------------
# Os modelos foram previamente treinados e validados fora deste notebook.
# Aqui, eles são utilizados exclusivamente para avaliação metamórfica.

try:
    modelos = {
        "ethanol": {
            "afr": joblib.load(
                Path(
                    r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/"
                    r"Conect2ai/MDPI2023-pollution/models/"
                    r"LGBMRegressor_ethanol_afr.pkl"
                )
            ),
            "maf": joblib.load(
                Path(
                    r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/"
                    r"Conect2ai/MDPI2023-pollution/models/"
                    r"XGBRegressor_ethanol_maf.pkl"
                )
            ),
        },
        "gasoline": {
            "afr": joblib.load(
                Path(
                    r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/"
                    r"Conect2ai/MDPI2023-pollution/models/"
                    r"LGBMRegressor_gasoline_afr.pkl"
                )
            ),
            "maf": joblib.load(
                Path(
                    r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/"
                    r"Conect2ai/MDPI2023-pollution/models/"
                    r"XGBRegressor_gasoline_maf.pkl"
                )
            ),
        },
    }

    print("Modelos de IA carregados com sucesso.")

except FileNotFoundError as e:
    print(f"ERRO: Arquivos de modelo não encontrados. {e}")
    raise

# ------------------------------------------------------------------------------
# 🔧 ETAPA 2: FUNÇÕES DE SUPORTE
# ------------------------------------------------------------------------------

def compute_co2(afr, maf, fuel_type):
    """
    Calcula a taxa de emissão de CO₂ a partir de AFR e MAF,
    considerando o tipo de combustível.
    """
    afr = np.asarray(afr)
    maf = np.asarray(maf)

    carbon_fraction = {
        "ethanol": 0.5217,
        "gasoline": 0.8571
    }

    fuel_rate = np.divide(
        maf,
        afr,
        out=np.zeros_like(maf, dtype=float),
        where=afr != 0
    )

    return fuel_rate * carbon_fraction[fuel_type] * (44.0 / 12.0)


def encontrar_coluna(df, alternativas):
    """
    Localiza dinamicamente colunas equivalentes,
    lidando com variações de nomenclatura nos datasets.
    """
    df.columns = df.columns.str.strip()
    for alt in alternativas:
        if alt in df.columns:
            return df[alt]
    return None


def pearson_safe(x, y):
    """
    Calcula a correlação de Pearson de forma robusta,
    ignorando NaNs e evitando exceções.
    """
    x = np.asarray(x)
    y = np.asarray(y)

    mask = ~np.isnan(x) & ~np.isnan(y)
    if mask.sum() < 2:
        return np.nan

    try:
        return pearsonr(x[mask], y[mask])[0]
    except Exception:
        return np.nan


# ------------------------------------------------------------------------------
# 🔬 FUNÇÃO PRINCIPAL DE TESTE – NÍVEL 2
# ------------------------------------------------------------------------------

def executar_teste_plotly(
    df_original,
    df_modificado,
    modelo_afr,
    modelo_maf,
    fuel,
    nome_mr,
    co2_real=None
):
    features = ["Latitude", "Longitude", "Speed(OBD)(km/h)", "Acceleration"]

    for feature in features:
        if feature not in df_original.columns:
            df_original[feature] = 0
        if feature not in df_modificado.columns:
            df_modificado[feature] = 0

    # ------------------------------------------------------------------
    # Prediction (Baseline)
    # ------------------------------------------------------------------
    afr_baseline = modelo_afr.predict(df_original[features])
    maf_baseline = modelo_maf.predict(df_original[features])
    co2_baseline = compute_co2(afr_baseline, maf_baseline, fuel)

    # ------------------------------------------------------------------
    # Prediction (Metamorphic)
    # ------------------------------------------------------------------
    afr_metamorphic = modelo_afr.predict(df_modificado[features])
    maf_metamorphic = modelo_maf.predict(df_modificado[features])
    co2_metamorphic = compute_co2(afr_metamorphic, maf_metamorphic, fuel)

    # ------------------------------------------------------------------
    # Métricas quantitativas
    # ------------------------------------------------------------------
    rmse = mean_squared_error(co2_baseline, co2_metamorphic, squared=False)

    pearson_baseline_mr = pearson_safe(co2_baseline, co2_metamorphic)
    pearson_gt_baseline = pearson_safe(co2_real, co2_baseline) if co2_real is not None else np.nan
    pearson_gt_mr = pearson_safe(co2_real, co2_metamorphic) if co2_real is not None else np.nan

    # ------------------------------------------------------------------
    # Log textual
    # ------------------------------------------------------------------
    print(f"\n{'='*80}")
    print(f"🔁 {nome_mr} ({fuel.upper()})")
    print("Metamorphic Test – Level 2 (Causal Coherence)")
    print(
        f"RMSE (Baseline × Metamorphic): {rmse:.4f} | "
        f"Pearson (Baseline × Metamorphic): {pearson_baseline_mr:.3f} | "
        f"Pearson (Ground Truth × Baseline): {pearson_gt_baseline:.3f} | "
        f"Pearson (Ground Truth × Metamorphic): {pearson_gt_mr:.3f}"
    )
    print(f"{'='*80}")

    # ------------------------------------------------------------------
    # Tabela ilustrativa (primeiras 10 amostras)
    # ------------------------------------------------------------------
    tabela = pd.concat(
        [
            df_original[features].iloc[:10].add_suffix("_Baseline"),
            pd.DataFrame({" ": ["→"] * 10}),
            df_modificado[features].iloc[:10].add_suffix("_Metamorphic"),
            pd.DataFrame({"  ": [" "] * 10}),
            pd.DataFrame({
                "CO₂_Ground_Truth": co2_real[:10] if co2_real is not None else "N/A",
                "CO₂_Pred_Baseline": co2_baseline[:10],
                "CO₂_Pred_Metamorphic": co2_metamorphic[:10]
            }),
        ],
        axis=1
    ).fillna("N/A")

    display(
        tabela.style.set_caption(
            f"Data comparison – {nome_mr} ({fuel.upper()})"
        )
    )

    # ------------------------------------------------------------------
    # Gráfico
    # ------------------------------------------------------------------
    fig = go.Figure()

    if co2_real is not None:
        fig.add_trace(
            go.Scatter(
                y=co2_real[:200],
                mode="lines",
                name="Ground Truth",
                line=dict(dash="dot", color="gray"),
            )
        )

    fig.add_trace(
        go.Scatter(
            y=co2_baseline[:200],
            mode="lines",
            name="Prediction (Baseline)",
            line=dict(dash="dash", color="blue"),
        )
    )

    fig.add_trace(
        go.Scatter(
            y=co2_metamorphic[:200],
            mode="lines",
            name="Prediction (Metamorphic)",
            line=dict(color="red"),
        )
    )

    fig.update_layout(
        title=dict(
            text=(
                f"CO₂ Comparison (200 Samples) – {nome_mr} – {fuel.upper()}<br>"
                f"RMSE={rmse:.3f} | "
                f"Pearson(Baseline×Metamorphic)={pearson_baseline_mr:.3f}"
            ),
            x=0.5,
        ),
        xaxis_title="Samples",
        yaxis_title="CO₂ (g/s)",
        template="plotly_white",
    )

    fig.show()


# ------------------------------------------------------------------------------
# 🔁 ETAPA 3: METAMORPHIC TEST CASES – LEVEL 2
# ------------------------------------------------------------------------------

testes_metamorficos_n2 = {
    "CT_2A_001 – [Multiplicative] Speed Scaling (×20)": lambda df: df.copy().assign(
        **{"Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"] * 20}
    ),

    "CT_2A_002 – [Multiplicative] Speed Scaling (×40)": lambda df: df.copy().assign(
        **{"Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"] * 40}
    ),


    "CT_2A_003 – [Multiplicative] Speed & Acceleration Scaling (×10)": 
        lambda df: df.copy().assign(
            **{
                "Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"] * 10,
                "Acceleration": df["Acceleration"] * 10
            }
        ),

    "CT_2A_004 – [Multiplicative] Speed & Acceleration Scaling (×20)": 
        lambda df: df.copy().assign(
            **{
                "Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"] * 20,
                "Acceleration": df["Acceleration"] * 20
            }
        ),

    "CT_2A_005 – [Multiplicative] Speed & Acceleration Scaling (×30)": 
        lambda df: df.copy().assign(
            **{
                "Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"] * 30,
                "Acceleration": df["Acceleration"] * 30
            }
        ),        


    "CT_2B_001 – [Additive] Coordinate Translation (+1.0)": lambda df: df.copy().assign(
        Latitude=df["Latitude"] + 1.0,
        Longitude=df["Longitude"] + 1.0
    ),

    "CT_2B_002 – [Additive] Coordinate Translation (+5.0)": lambda df: df.copy().assign(
        Latitude=df["Latitude"] + 5.0,
        Longitude=df["Longitude"] + 5.0
    ),

    "CT_2C_001 – [Inversion] Acceleration Sign Inversion": lambda df: df.copy().assign(
        Acceleration=df["Acceleration"] * -1
    ),

    "CT_2D_001 – [Temporal] Speed Lag (lag = 1)": lambda df: df.copy().assign(
        **{"Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"].shift(1)}
    ),

    "CT_2D_002 – [Temporal] Speed Lag (lag = 5)": lambda df: df.copy().assign(
        **{"Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"].shift(5)}
    ),

    "CT_2D_003 – [Temporal] Speed Lag (lag = 10)": lambda df: df.copy().assign(
        **{"Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"].shift(10)}
    ),

    "CT_2D_004 – [Temporal] Speed Lag (lag = 25)": lambda df: df.copy().assign(
        **{"Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"].shift(25)}
    ),

    "CT_2D_005 – [Temporal] Speed Lag (lag = 50)": lambda df: df.copy().assign(
        **{"Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"].shift(50)}
    ),
}


# ------------------------------------------------------------------------------
# 🧲 ETAPA 4: TEST EXECUTION – LEVEL 2
# ------------------------------------------------------------------------------

print("=" * 80)
print("STARTING LEVEL 2 – METAMORPHIC TEST EXECUTION (CAUSAL COHERENCE)")
print("=" * 80)

# ------------------------------------------------------------------
# Ground Truth computation
# ------------------------------------------------------------------
afr_real_ethanol = encontrar_coluna(
    df_ethanol,
    ["AirFuelRatio(Commanded)(:1)", "Air Fuel Ratio(Commanded)(:1)"]
)
maf_real_ethanol = encontrar_coluna(
    df_ethanol,
    ["MassAirFlowRate(g/s)", "Mass Air Flow Rate(g/s)"]
)

co2_real_ethanol = (
    compute_co2(afr_real_ethanol.values, maf_real_ethanol.values, "ethanol")
    if afr_real_ethanol is not None and maf_real_ethanol is not None
    else None
)

afr_real_gasoline = encontrar_coluna(
    df_gasoline,
    ["AirFuelRatio(Commanded)(:1)", "Air Fuel Ratio(Commanded)(:1)"]
)
maf_real_gasoline = encontrar_coluna(
    df_gasoline,
    ["MassAirFlowRate(g/s)", "Mass Air Flow Rate(g/s)"]
)

co2_real_gasoline = (
    compute_co2(afr_real_gasoline.values, maf_real_gasoline.values, "gasoline")
    if afr_real_gasoline is not None and maf_real_gasoline is not None
    else None
)

ground_truth_values = {
    "ethanol": co2_real_ethanol,
    "gasoline": co2_real_gasoline
}

# ------------------------------------------------------------------
# Test execution loop
# ------------------------------------------------------------------
for nome_mr, transformacao in testes_metamorficos_n2.items():
    for fuel, df_original in [
        ("ethanol", df_ethanol),
        ("gasoline", df_gasoline)
    ]:
        df_modificado = transformacao(df_original.copy())

        executar_teste_plotly(
            df_original=df_original,
            df_modificado=df_modificado,
            modelo_afr=modelos[fuel]["afr"],
            modelo_maf=modelos[fuel]["maf"],
            fuel=fuel,
            nome_mr=nome_mr,
            co2_real=ground_truth_values[fuel]
        )



Arquivos de dados carregados com sucesso.
Modelos de IA carregados com sucesso.
STARTING LEVEL 2 – METAMORPHIC TEST EXECUTION (CAUSAL COHERENCE)

🔁 CT_2A_001 – [Multiplicative] Speed Scaling (×20) (ETHANOL)
Metamorphic Test – Level 2 (Causal Coherence)
RMSE (Baseline × Metamorphic): 0.6804 | Pearson (Baseline × Metamorphic): 0.673 | Pearson (Ground Truth × Baseline): 0.821 | Pearson (Ground Truth × Metamorphic): 0.575


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,,CO₂_Ground_Truth,CO₂_Pred_Baseline,CO₂_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,60,0.000000,,0.642839,0.742631,1.006023
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,100,0.555556,,0.653249,0.771505,1.623527
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,100,0.000000,,0.653249,0.771505,1.579490
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,60,-0.555556,,0.679275,0.659013,0.914030
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,0,-0.833333,,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0,0.000000,,0.610306,0.706858,0.706858
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0,0.000000,,0.490587,0.706858,0.706858
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,60,0.833333,,1.090483,1.137803,1.000498
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,60,0.000000,,1.090483,1.137803,0.924211
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,60,0.000000,,0.921315,1.137803,0.924211



🔁 CT_2A_001 – [Multiplicative] Speed Scaling (×20) (GASOLINE)
Metamorphic Test – Level 2 (Causal Coherence)
RMSE (Baseline × Metamorphic): 0.6509 | Pearson (Baseline × Metamorphic): 0.827 | Pearson (Ground Truth × Baseline): 0.807 | Pearson (Ground Truth × Metamorphic): 0.680


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,,CO₂_Ground_Truth,CO₂_Pred_Baseline,CO₂_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,100,0.000000,,1.002671,1.456781,3.189422
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,80,-0.277778,,1.013360,1.016708,2.051800
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,100,0.277778,,2.088720,1.456781,3.550872
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,140,0.555556,,2.261889,1.996380,3.550872
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,180,0.555556,,2.499195,2.400486,3.046464
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,240,0.833333,,2.606089,2.400486,3.046464
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,300,0.833333,,3.070012,2.148721,2.948598
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,360,0.833333,,2.445747,1.470940,2.285238
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,360,0.000000,,1.073221,1.588029,2.052619
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,320,-0.555556,,0.562265,1.085921,1.591138



🔁 CT_2A_002 – [Multiplicative] Speed Scaling (×40) (ETHANOL)
Metamorphic Test – Level 2 (Causal Coherence)
RMSE (Baseline × Metamorphic): 0.6885 | Pearson (Baseline × Metamorphic): 0.669 | Pearson (Ground Truth × Baseline): 0.821 | Pearson (Ground Truth × Metamorphic): 0.571


...

,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,,CO₂_Ground_Truth,CO₂_Pred_Baseline,CO₂_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,N/A,0.000000,,1.002671,1.456781,3.653032
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,N/A,-0.277778,,1.013360,1.016708,3.031917
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,N/A,0.277778,,2.088720,1.456781,3.653032
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,N/A,0.555556,,2.261889,1.996380,3.653032
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,N/A,0.555556,,2.499195,2.400486,3.134111
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,N/A,0.833333,,2.606089,2.400486,3.134111
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,N/A,0.833333,,3.070012,2.148721,3.033430
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,N/A,0.833333,,2.445747,1.470940,2.350985
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,N/A,0.000000,,1.073221,1.588029,2.350985
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,N/A,-0.555556,,0.562265,1.085921,2.351204



🔁 CT_2D_005 – [Temporal] Speed Lag (lag = 50) (ETHANOL)
Metamorphic Test – Level 2 (Causal Coherence)
RMSE (Baseline × Metamorphic): 0.4265 | Pearson (Baseline × Metamorphic): 0.488 | Pearson (Ground Truth × Baseline): 0.821 | Pearson (Ground Truth × Metamorphic): 0.391


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,,CO₂_Ground_Truth,CO₂_Pred_Baseline,CO₂_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,N/A,0.000000,,0.642839,0.742631,1.573511
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,N/A,0.555556,,0.653249,0.771505,1.573511
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,N/A,0.000000,,0.653249,0.771505,1.573511
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,N/A,-0.555556,,0.679275,0.659013,1.428636
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,N/A,-0.833333,,0.610306,0.667689,1.366802
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,N/A,0.000000,,0.610306,0.706858,1.514831
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,N/A,0.000000,,0.490587,0.706858,1.514831
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,N/A,0.833333,,1.090483,1.137803,1.514831
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,N/A,0.000000,,1.090483,1.137803,1.514831
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,N/A,0.000000,,0.921315,1.137803,1.514831



🔁 CT_2D_005 – [Temporal] Speed Lag (lag = 50) (GASOLINE)
Metamorphic Test – Level 2 (Causal Coherence)
RMSE (Baseline × Metamorphic): 0.5931 | Pearson (Baseline × Metamorphic): 0.690 | Pearson (Ground Truth × Baseline): 0.807 | Pearson (Ground Truth × Metamorphic): 0.547


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,,CO₂_Ground_Truth,CO₂_Pred_Baseline,CO₂_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,N/A,0.000000,,1.002671,1.456781,3.653032
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,N/A,-0.277778,,1.013360,1.016708,3.031917
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,N/A,0.277778,,2.088720,1.456781,3.653032
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,N/A,0.555556,,2.261889,1.996380,3.653032
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,N/A,0.555556,,2.499195,2.400486,3.134111
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,N/A,0.833333,,2.606089,2.400486,3.134111
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,N/A,0.833333,,3.070012,2.148721,3.033430
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,N/A,0.833333,,2.445747,1.470940,2.350985
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,N/A,0.000000,,1.073221,1.588029,2.350985
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,N/A,-0.555556,,0.562265,1.085921,2.351204
